# Curvilinear grids


Parcels supports [curvilinear grids](https://www.nemo-ocean.eu/doc/node108.html) such as those used in the [NEMO models](https://www.nemo-ocean.eu/).

We will be using the example dataset `NemoCurvilinear_data`. These fields are a purely zonal flow on an aqua-planet (so zonal-velocity is 1 m/s and meridional-velocity is 0 m/s everywhere, and no land). However, because of the curvilinear grid, the `U` and `V` fields vary for the rotated gridcells north of 20N.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

import parcels

We can create a `FieldSet` just like we do for normal grids.
Note that NEMO is discretised on a C-grid. U and V velocities are not located on the same nodes (see https://www.nemo-ocean.eu/doc/node19.html ).

```
 __V1__
|      |
U0     U1
|__V0__|
```

To interpolate U, V velocities on the C-grid, Parcels needs to read the f-nodes, which are located on the corners of the cells (for indexing details: https://www.nemo-ocean.eu/doc/img360.png).

```{note}
TODO: add link to grid indexing explanation once implemented in v4
```


In [ ]:
data_folder = parcels.download_example_dataset("NemoCurvilinear_data")
files = data_folder.glob("*.nc4")
ds_fields = xr.open_mfdataset(
    files, combine="nested", data_vars="minimal", coords="minimal", compat="override"
)


# TODO: replace manual fieldset creation with FieldSet.from_nemo() once available
ds_fields = (
    ds_fields.isel(time_counter=0, drop=True)
    .isel(time=0, drop=True)
    .isel(z_a=0, drop=True)
    .rename({"glamf": "lon", "gphif": "lat", "z": "depth"})
)

import xgcm

xgcm_grid = xgcm.Grid(
    ds_fields,
    coords={
        "X": {"left": "x"},
        "Y": {"left": "y"},
    },
    periodic=False,
    autoparse_metadata=False,
)
grid = parcels.XGrid(xgcm_grid, mesh="spherical")

U = parcels.Field(
    "U", ds_fields["U"], grid, interp_method=parcels.interpolators.XLinear
)
V = parcels.Field(
    "V", ds_fields["V"], grid, interp_method=parcels.interpolators.XLinear
)
U.units = parcels.GeographicPolar()
V.units = (
    parcels.GeographicPolar()
)  # U and V need GeographicPolar for  C-Grid interpolation to work correctly
UV = parcels.VectorField(
    "UV", U, V, vector_interp_method=parcels.interpolators.CGrid_Velocity
)
fieldset = parcels.FieldSet([U, V, UV])

And we can plot the `U` field.


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 4))
pc1 = ds_fields.U.plot(cmap="viridis", ax=ax[0], vmin=0)
pc2 = ax[1].pcolormesh(
    fieldset.U.grid.lon,
    fieldset.U.grid.lat,
    fieldset.U.data[0, 0, :, :],
    vmin=0,
    vmax=1,
)
ax[1].set_ylabel("Latitude [deg N]")
ax[1].set_xlabel("Longitude [deg E]")
plt.colorbar(pc2, label="U", ax=ax[1])
plt.tight_layout()
plt.show()

As you see above, the `U` field indeed is 1 m/s south of 20N, but varies with longitude and latitude north of that. We can confirm that Parcels will take care to rotate the `U` and `V` fields by doing a field evaluation at (60N, 50E):

In [ ]:
u, v = fieldset.UV.eval(
    np.array([0]), np.array([0]), np.array([20]), np.array([50]), applyConversion=False
)  # do not convert m/s to deg/s
print(f"(u, v) = ({u[0]:.3f}, {v[0]:.3f})")
assert np.isclose(u, 1.0, atol=1e-3)

Now we can run particles as normal.

In [ ]:
npart = 20
lonp = 30 * np.ones(npart)
latp = np.linspace(-70, 88, npart)
runtime = np.timedelta64(40, "D")

pset = parcels.ParticleSet(fieldset, lon=lonp, lat=latp)
pfile = parcels.ParticleFile(
    store="output_curvilinear.zarr", outputdt=np.timedelta64(1, "D")
)

pset.execute(
    [parcels.kernels.AdvectionEE],
    runtime=runtime,
    dt=np.timedelta64(1, "D"),
    output_file=pfile,
)
np.testing.assert_allclose(pset.lat, latp, atol=1e-1)

And then we can plot these trajectories. As expected, all trajectories go exactly zonal and due to the curvature of the earth, ones at higher latitude move more degrees eastward (even though the distance in km is equal for all particles). These particles at high latitudes cross the antimeridian (180 deg E) and keep going east.


In [ ]:
ds = xr.open_zarr("output_curvilinear.zarr")

plt.plot(ds.lon.T, ds.lat.T, ".-")
plt.vlines(np.arange(-180, 901, 360), -90, 90, color="r", label="antimeridian")
plt.ylabel("Latitude [deg N]")
plt.xlabel("Longitude [deg E]")
plt.xticks(np.arange(-180, 901, 90))
plt.legend(loc="lower right")
plt.show()

If we want the `particles.lon` to stay within `[-180,180]` (or `[0,360]`), we can either do this in post-processing, or add a periodic boundary Kernel:

In [ ]:
# post processing
ds["lon"] = ds["lon"] % 360
ds["lon"] = ds["lon"].where(ds["lon"] <= 180, ds["lon"] - 360)

In [ ]:
# with a Kernel
def periodicBC(particles, fieldset):  # pragma: no cover
    particles.dlon = np.where(
        particles.lon + particles.dlon > 180, particles.dlon - 360, particles.dlon
    )


pset = parcels.ParticleSet(fieldset, lon=lonp, lat=latp)
pfile = parcels.ParticleFile(
    store="output_curvilinear_periodic.zarr", outputdt=np.timedelta64(1, "D")
)

pset.execute(
    [parcels.kernels.AdvectionEE, periodicBC],
    runtime=runtime,
    dt=np.timedelta64(1, "D"),
    output_file=pfile,
)

In [ ]:
ds_periodic = xr.open_zarr("output_curvilinear_periodic.zarr")

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].plot(ds.lon.T, ds.lat.T, ".-")
ax[0].vlines(np.arange(-180, 360, 360), -90, 90, color="r", label="antimeridian")
ax[0].set_ylabel("Latitude [deg N]")
ax[0].set_xlabel("Longitude [deg E]")
ax[0].set_xticks(np.arange(-180, 181, 45))
ax[0].set_title("in post processing")
ax[0].legend(loc="lower center")

ax[1].plot(ds_periodic.lon.T, ds_periodic.lat.T, ".-")
ax[1].vlines(np.arange(-180, 360, 360), -90, 90, color="r", label="antimeridian")
ax[1].set_ylabel("Latitude [deg N]")
ax[1].set_xlabel("Longitude [deg E]")
ax[1].set_xticks(np.arange(-180, 181, 45))
ax[1].set_title("with periodic Kernel")
ax[1].legend(loc="lower center")

plt.show()